In [1]:
import logging

In [3]:
class Snake_log:
    def __init__(self, head_position, food_position, direction, game_size, snake_positions, reverse_commands, ip, commands_history, score, play_mode, steps):
        self.head_x_pos = head_position[0]
        self.head_y_pos = head_position[1]
        self.food_x_pos = food_position[0]
        self.food_y_pos = food_position[1]
        self.direction = direction
        self.game_size = game_size
        self.snake_positions = snake_positions
        self.reverse_commands = reverse_commands
        self.directions_left = []
        self.ip = ip
        self.history = commands_history
        self.is_eaten = False
        self.logger = None
        self.ch = None
        self.fh = None
        self.score = score
        self.play_mode = play_mode
        self.steps = steps
        self.params = {}
        
    def update_logger(self):
        self.fh.setFormatter(self.formatter)
        self.ch.setFormatter(self.formatter)
        
    def init_logger(self) :
        #On instancie un logger
        self.logger = logging.getLogger('snake_logger')
        self.logger.setLevel(logging.DEBUG)
        
        #On créé des handlers pour renvoyer les logs où on le desire
        self.ch = logging.StreamHandler()
        self.ch.setLevel(logging.INFO)
        
        #Vérifie que des handlers existent déjà, si oui, on les supprime
        self.fh = logging.FileHandler(f'snake_logs.log') #choose a file name
        self.fh.setLevel(logging.INFO)
        
        #On ajoute nos handlers instanciés 
        if (self.logger.hasHandlers()):
            self.logger.handlers.clear()
            
        self.logger.addHandler(self.ch)
        self.logger.addHandler(self.fh)
        
        self.history.append(self.direction)
        
    def logging_utils(self, game_status) :

        self.params = {
            "clientip" :  self.ip,
            "head_x" : self.head_x_pos,
            "head_y" : self.head_y_pos,
            "food_x" : self.food_x_pos,
            "food_y" : self.food_y_pos,
            "score" : self.score,
            "play_mode" : self.play_mode,
            "steps" : self.steps,
            "direction" : self.direction  
        }
        
        #Changement de direction
        if self.history[-2] != self.direction :
            self.params["logcat"] = "DIRECTION"
            self.formatter = logging.Formatter('[%(clientip)s] - %(logcat)s - IA ID : %(play_mode)s - %(asctime)s - %(levelname)s - Direction : %(direction)s - %(message)s')
            self.update_logger()
            self.logger.info(f"Direction changed : {self.direction}", extra=self.params)
            
        #Direction opposée
        if self.history[-2] == self.reverse_commands[self.direction] :
            self.params["logcat"] = "OPOSITE_DETECTED"
            self.formatter = logging.Formatter('[%(clientip)s] - %(logcat)s - IA ID : %(play_mode)s - %(asctime)s - %(levelname)s - Snake position : (%(head_x)s, %(head_y)s) - %(message)s')
            self.update_logger()
            self.logger.critical(f"Opposite direction detected ! Last direction : {self.history[-2]} - Direction :{self.direction}", extra=self.params)
        
        #La pomme est mangée
        if game_status["is_eaten"]:
            self.params["logcat"] = "APPLE_IS_EATEN"
            self.formatter = logging.Formatter('[%(clientip)s] - %(logcat)s - IA ID : %(play_mode)s - %(asctime)s - %(levelname)s - Apple position : (%(food_x)s, %(food_y)s) - %(message)s')
            self.update_logger()
            self.logger.info(f"Apple is eaten at {self.food_x_pos, self.food_y_pos}!", extra=self.params)
        
        #Le snake se mange lui même
        if game_status["is_eat_itself"]:
            self.params["logcat"] = "GAMEOVER_EAT_ITSELF"
            self.formatter = logging.Formatter('[%(clientip)s] - %(logcat)s - IA ID : %(play_mode)s - %(asctime)s - %(levelname)s - Snake position : (%(head_x)s, %(head_y)s) - Score : %(score)s - Steps : %(steps)s - %(message)s')
            self.update_logger()
            self.logger.info(f"GAME OVER !", extra=self.params)
            
        #Le snake va dans le mur
        if game_status["is_touched_wall"]:
            self.params["logcat"] = "GAMEOVER_WALL"
            self.formatter = logging.Formatter('[%(clientip)s] - %(logcat)s - IA ID : %(play_mode)s - %(asctime)s - %(levelname)s - Snake position : (%(head_x)s, %(head_y)s) - Score : %(score)s - Steps : %(steps)s - %(message)s')
            self.update_logger()
            self.logger.info(f"GAME OVER !", extra=self.params)